In [1]:
import os
import sys

import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.insert(0, os.getcwd() + '/../../')
sys.path.insert(0, os.getcwd() + '/../../abm/')
sys.path.insert(0, os.getcwd() + '/../../simulation/')
sys.path.insert(0, os.getcwd() + '/../../mesa/')

from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.UserParam import UserSettableParameter

from simulation.geospatial.model import GeoSimulationEnvironment
from abm.space.geospace import GeoSpaceEnvironment
from abm.space.geojson.constants import BARMM_GEOJSON

from abm.utils.modules import DataManager
from abm.resources.util import DATA_DROP_CSV_FILE, BARMM_DATA_LOC
from simulation.grid.model import GridSimulationEnvironment

from abm.utils.collectors.total_susceptible import total_susceptible
from abm.utils.collectors.total_infected import total_infected
from abm.utils.collectors.total_deaths import total_deaths
from abm.utils.collectors.total_recovered import total_recovered
from abm.utils.collectors.total_vaccinated import total_vaccinated

from abm.utils.collectors.count_age_infected import *
from abm.utils.collectors.count_age_exposed import *
from abm.utils.collectors.count_age_died import *
from abm.utils.collectors.count_age_recovered import *

%matplotlib inline

plt.style.use('ggplot')

plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

warnings.simplefilter('ignore')

/home/joma.minoza/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (10,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
simulation_model        = GeoSimulationEnvironment
center_coords           = [5.986803, 122.1695733]
locations               = ["BASILAN", "LANAO DEL SUR", "MAGUINDANAO", "SULU", "TAWI-TAWI"]
age_distributions       = [
                            1037200,   # 0 - 9 
                            931800,    # 10 - 19 
                            725000,    # 20 - 29 
                            629100,    # 30 - 39
                            453200,    # 40 - 49
                            296000,    # 50 - 59
                            173700,    # 60 - 69
                            68800,     # 70 - 79
                            15500      # 80 and up
                        ]

population              = [
                            385300,         # Basilan
                            1225700,        # Lanao Del Sur
                            1391100,        # Maguindanao
                            839500,         # Sulu
                            489100          # Tawi-Tawi
                        ]

vaccine_prioritization_weights = {
    "person_health_social_work":        [0.10, 0.32, 0.09, 0.37, 0.11],
    "person_prof_tech":                 [0.06, 0.37, 0.37, 0.12, 0.10],
    "person_admin_support":             [0.01, 0.75, 0.08, 0.10, 0.06],
    "person_education":                 [0.03, 0.77, 0.12, 0.04, 0.05],
    "person_agriculture":               [0.10, 0.75, 0.06, 0.06, 0.03],
    "elderly":                          [0.09, 0.28, 0.32, 0.19, 0.11],
    "social_household_4ps_active":      [0.066, 0.239, 0.306, 0.330, 0.059],
    "social_bayanihan_grant":           [0.08, 0.33, 0.31, 0.17, 0.11],
    "economics_tourists":               [0.1352, 0.1919, 0.3569, 0.0004, 0.3156],
    "economics_marine_fisheries":       [0.03, 0.23, 0.16, 0.13, 0.45],
    "economics_volume_fisheries":       [0.02, 0.02, 0.16, 0.36, 0.44],
    "economics_livestock_inventory":    [0.28, 0.31, 0.24, 0.10, 0.06],
    "economics_volume_corn":            [0.00009, 0.33832, 0.65840, 0.00129, 0.00189]
}

performance_factor       = 0.5
scale                    = 1000 * performance_factor  
vaccines_available       = 5000 / performance_factor # times scale (actual number for vaccine doses) 

data_manager            = DataManager(DATA_DROP_CSV_FILE, BARMM_DATA_LOC, "ProvRes", locations, age_distributions)
environment             = GeoSpaceEnvironment(simulation_model, BARMM_GEOJSON, "PROVINCE", locations, center_coords, population, data_manager, "ProvRes", vaccine_prioritization_weights, vaccines_available)
environment.scale       = scale

simulation_model.grid   = environment.get_geospace()

Initializing the data ...
Updating LOCATION_DATA field...
New Data: [{'LOCATION_NAME': 'Basilan', 'POPULATION': 385300, 'DATA': {'susceptible': 0.9991123799636646, 'infected': 0.0006047235920062289, 'recovered': 0.00026732416298987804, 'dead': 1.5572281339216195e-05}, 'VACCINE_ALLOCATION': {'age_00-09': {'min': 0, 'max': 9, 'value': 0}, 'age_10-19': {'min': 10, 'max': 19, 'value': 0}, 'age_20-29': {'min': 20, 'max': 29, 'value': 0}, 'age_30-39': {'min': 30, 'max': 39, 'value': 0}, 'age_40-49': {'min': 40, 'max': 49, 'value': 0}, 'age_50-59': {'min': 50, 'max': 59, 'value': 0}, 'age_60-69': {'min': 60, 'max': 69, 'value': 0}, 'age_70-79': {'min': 70, 'max': 79, 'value': 0}, 'age_80-up': {'min': 80, 'max': 100, 'value': 0}}, 'SUSCEPTIBLE_AGENTS': {'age_00-09': {'min': 0, 'max': 9, 'value': 0}, 'age_10-19': {'min': 10, 'max': 19, 'value': 0}, 'age_20-29': {'min': 20, 'max': 29, 'value': 0}, 'age_30-39': {'min': 30, 'max': 39, 'value': 0}, 'age_40-49': {'min': 40, 'max': 49, 'value': 0}, '

In [3]:
default_scenario_model_params = {
    "scenarios":                     "No Vaccination",
    "vaccination_implementation":    "After 0 Days",
    "vaccine_hesitancy":             0,
    "person_health_social_work":     True,
    "person_prof_tech":              True,
    "person_admin_support":          True,
    "person_education":              True,
    "person_agriculture":            True,
    "elderly":                       False,
    "social_household_4ps_active":   False,
    "social_bayanihan_grant":        False,
    "economics_tourists":            False,
    "economics_marine_fisheries":    False,
    "economics_volume_fisheries":    False,
    "economics_livestock_inventory": False,
    "economics_volume_corn":         False,
    "incubation_period":             7,
    "recovery_period":               14,
    "transmission_rate":             0.213,  
    "viral_load_probability":        0,
    "wearing_masks":                 0.53,
    "social_distance_limit":         0.75, # Social Distance Limit 
    "natural_immunity":              0.18, # VACCINATION COVERAGE OF FILIPINO CHILDREN for ARMM
    "exercise":                      0.328, # PHYSICAL ACTIVITY AND DIETARY HABITS OF FILIPINO
    "preexisting_conditions":        0.22,
    "minority_restrictions":         15,
    "adult_restrictions":            65,   
    "environment":                   environment
}

default_vaccination_model_params  = default_scenario_model_params.copy()
default_vaccination_model_params["scenarios"] = "With Vaccination"
default_vaccination_model_params["vaccine_hesitancy"] = 0.5

social_vaccination_model_params = default_vaccination_model_params.copy()
social_vaccination_model_params["social_household_4ps_active"] = True
social_vaccination_model_params["social_bayanihan_grant"] = True

mobility_vaccination_model_params = default_vaccination_model_params.copy()
mobility_vaccination_model_params["economics_tourists"] = True
mobility_vaccination_model_params["economics_marine_fisheries"] = True
mobility_vaccination_model_params["economics_volume_fisheries"] = True
mobility_vaccination_model_params["economics_livestock_inventory"] = True
mobility_vaccination_model_params["economics_volume_corn"] = True

elderly_vaccination_model_params = default_vaccination_model_params.copy()
elderly_vaccination_model_params["elderly"] = True

all_configuration_vaccination_model_params = default_vaccination_model_params.copy()
all_configuration_vaccination_model_params["social_household_4ps_active"] = True
all_configuration_vaccination_model_params["social_bayanihan_grant"] = True
all_configuration_vaccination_model_params["economics_tourists"] = True
all_configuration_vaccination_model_params["economics_marine_fisheries"] = True
all_configuration_vaccination_model_params["economics_volume_fisheries"] = True
all_configuration_vaccination_model_params["economics_livestock_inventory"] = True
all_configuration_vaccination_model_params["economics_volume_corn"] = True
all_configuration_vaccination_model_params["elderly"] = True

In [4]:
number_of_iterations = 20
number_of_steps      = 240

parent_dir = os.getcwd()
scenario   = "results/default_vaccination_model_50_hesistancy"

path = os.path.join(parent_dir, scenario) 
os.makedirs(path,  exist_ok=True)

agents_infected_path   = os.path.join(path, "agents_infected")
agents_exposed_path    = os.path.join(path, "agents_exposed")
agents_recovered_path  = os.path.join(path, "agents_recovered")
agents_vaccinated_path = os.path.join(path, "agents_vaccinated")
agents_summary_path = os.path.join(path, "agents_summary")

os.makedirs(agents_infected_path,  exist_ok=True)
os.makedirs(agents_exposed_path,   exist_ok=True)
os.makedirs(agents_recovered_path, exist_ok=True)
os.makedirs(agents_vaccinated_path,exist_ok=True)
os.makedirs(agents_summary_path,   exist_ok=True)

In [5]:
for iteration in range(number_of_iterations):
    default_scenario_model = simulation_model(**default_vaccination_model_params)
    while default_scenario_model.running and default_scenario_model.schedule.steps < number_of_steps:
        default_scenario_model.step()
    
    agents_infected_collector = default_scenario_model.agents_infected_collector.get_model_vars_dataframe()
    agents_infected_collector.to_csv("{0}/{1}.csv".format(agents_infected_path, iteration))
    
    agents_exposed_collector = default_scenario_model.agents_exposed_collector.get_model_vars_dataframe()
    agents_exposed_collector.to_csv("{0}/{1}.csv".format(agents_exposed_path, iteration))

    agents_died_collector = default_scenario_model.agents_died_collector.get_model_vars_dataframe()
    agents_died_collector.to_csv("{0}/{1}.csv".format(agents_infected_path, iteration))

    agents_recovered_collector = default_scenario_model.agents_recovered_collector.get_model_vars_dataframe()
    agents_recovered_collector.to_csv("{0}/{1}.csv".format(agents_recovered_path, iteration))

    agents_vaccinated_collector = default_scenario_model.agents_recovered_collector.get_model_vars_dataframe()
    agents_vaccinated_collector.to_csv("{0}/{1}.csv".format(agents_vaccinated_path, iteration))

    agents_summary_collector = default_scenario_model.summary_data_collector.get_model_vars_dataframe()
    agents_summary_collector.to_csv("{0}/{1}.csv".format(agents_summary_path, iteration))
    

Initializing Agents for Basilan
Basilan 769.916 0.466 0.20600000000000002 0.012
Initializing Agents for Lanao del Sur
Lanao del Sur 2449.02 2.096 0.2 0.08400000000000002
Initializing Agents for Maguindanao
Maguindanao 2780.41 1.7100000000000002 0.057999999999999996 0.022
Initializing Agents for Sulu
Sulu 1678.186 0.57 0.226 0.018
Initializing Agents for Tawi-Tawi
Tawi-Tawi 977.886 0.3 0.006 0.008
Updating LOCATION_DATA field...
New Data: [{'LOCATION_NAME': 'Basilan', 'POPULATION': 385300, 'DATA': {'susceptible': 0.9991123799636646, 'infected': 0.0006047235920062289, 'recovered': 0.00026732416298987804, 'dead': 1.5572281339216195e-05}, 'VACCINE_ALLOCATION': {'age_00-09': {'min': 0, 'max': 9, 'value': 0}, 'age_10-19': {'min': 10, 'max': 19, 'value': 0}, 'age_20-29': {'min': 20, 'max': 29, 'value': 0}, 'age_30-39': {'min': 30, 'max': 39, 'value': 0}, 'age_40-49': {'min': 40, 'max': 49, 'value': 0}, 'age_50-59': {'min': 50, 'max': 59, 'value': 0}, 'age_60-69': {'min': 60, 'max': 69, 'value